In [1]:
import itertools
import joblib
import pickle 

import networkx as nx
import numpy as np
import pandas as pd
import cvxpy as cp

import matplotlib.pyplot as plt
import matplotlib.pylab as pl
import seaborn as sns
import random 
import joblib

import numpy as np
from sklearn.linear_model import Lasso
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import mean_squared_error

import numpy as np
import networkx as nx

from src.CBN import CausalBayesianNetwork as CBN
import modularised_utils as mut
import Linear_Additive_Noise_Models as lanm
import operations as ops
import evaluation_utils as evut
import opt_utils as oput
import params

np.random.seed(0)

[KeOps] Warning : CUDA libraries not found or could not be loaded; Switching to CPU only.


In [2]:
M_base = joblib.load('batteries/scms/M_WMG_bins_5_avg_2.pkl')
M_abst = joblib.load('batteries/scms/M_LRCS_bins_5.pkl') 

In [3]:
# Base-level interventions (new style)
iota0 = None
iota1 = ops.Intervention({'CG': 75.})
iota2 = ops.Intervention({'CG': 110.})
iota3 = ops.Intervention({'CG': 180.})
iota4 = ops.Intervention({'CG': 200.})

# Abstract-level interventions (new style)
iota0_prime = None
iota1_prime = ops.Intervention({'CG': 75.})
iota2_prime = ops.Intervention({'CG': 100.})
iota3_prime = ops.Intervention({'CG': 200.})

# Mapping
omega = {
    iota0: iota0_prime,
    iota1: iota1_prime,
    iota2: iota2_prime,
    iota3: iota3_prime,
    iota4: iota3_prime
}

Ill = list(set(omega.keys()))
Ihl = list(set(omega.values()))


In [4]:
df_base = joblib.load('batteries/dfs/df_WMG_bins_5_avg_2.pkl')
df_abst = joblib.load('batteries/dfs/df_LRCS_bins_5.pkl')

In [5]:
df_base.drop(df_base.columns[[1,2]], axis=1, inplace=True)
df_base.replace({75:0, 110:1, 150:2, 170:3, 180:4, 200:5}, inplace=True)

df_abst.drop(df_abst.columns[[1]], axis=1, inplace=True)
df_abst.replace({75:0, 100:1, 200:2}, inplace=True)

In [6]:
# Rename columns to match graph
df_base = df_base.rename(columns={
    'binned ML_avg0': 'ML0',
    'binned ML_avg1': 'ML1'
})
# Rename columns to match graph
df_abst = df_abst.rename(columns={
    'Comma gap (µm)': 'CG', 'binned ML': 'ML'
})

In [7]:
# Gll = nx.DiGraph()
# Gll.add_nodes_from(M_base.nodes())
# Gll.add_edges_from(M_base.edges())
# Ghl = nx.DiGraph()
# Ghl.add_nodes_from(M_abst.nodes())
# Ghl.add_edges_from(M_abst.edges())

In [8]:
# # First, convert your data to numpy array if it's not already
# if isinstance(df_base, str):
#     # If df_base is a path to a file
#     df_base = np.loadtxt(df_base)  # or pd.read_csv(df_base).values
# elif isinstance(df_base, pd.DataFrame):
#     # If df_base is a pandas DataFrame
#     df_base = df_base.values
# else:
#     # Ensure it's a numpy array
#     df_base = np.array(df_base)

# # Now you can use get_coefficients
# ll_endogenous_coeff_dict = mut.get_coefficients(df_base, Gll)

# # Same for high-level data
# if isinstance(df_abst, str):
#     df_abst = np.loadtxt(df_abst)
# elif isinstance(df_abst, pd.DataFrame):
#     df_abst = df_abst.values
# else:
#     df_abst = np.array(df_abst)

# hl_endogenous_coeff_dict = mut.get_coefficients(df_abst, Ghl)

In [9]:
# U_ll, ll_mu_hat, ll_Sigma_hat = mut.lan_abduction(df_base, Gll, ll_endogenous_coeff_dict)
# U_hl, hl_mu_hat, hl_Sigma_hat = mut.lan_abduction(df_abst, Ghl, hl_endogenous_coeff_dict)

In [10]:
ll_coeffs = {}

for child in M_base.nodes():
    parents = M_base.get_parents(child)
    if not parents:
        continue  # skip root nodes
    X = df_base[parents].values
    y = df_base[child].values
    coef = np.linalg.lstsq(X, y, rcond=None)[0]
    for i, parent in enumerate(parents):
        ll_coeffs[(parent, child)] = coef[i]

Gll = CBN(list(ll_coeffs.keys()))

hl_coeffs = {}

for child in M_abst.nodes():
    parents = M_abst.get_parents(child)
    if not parents:
        continue  # skip root nodes
    X = df_abst[parents].values
    y = df_abst[child].values
    coef = np.linalg.lstsq(X, y, rcond=None)[0]
    for i, parent in enumerate(parents):
        hl_coeffs[(parent, child)] = coef[i]

Ghl = CBN(list(hl_coeffs.keys()))


In [11]:
min_samples = min(df_base.shape[0], df_abst.shape[0])
df_base = df_base[:min_samples]
df_abst = df_abst[:min_samples]

df_base= df_base.to_numpy()
df_abst= df_abst.to_numpy()

In [12]:
num_llsamples = df_base.shape[0]
num_hlsamples = df_abst.shape[0]
l = len(Gll.nodes())
h = len(Ghl.nodes())

In [13]:
U_ll_hat, mu_U_ll_hat, Sigma_U_ll_hat = mut.lan_abduction(df_base, Gll, ll_coeffs)
U_hl_hat, mu_U_hl_hat, Sigma_U_hl_hat = mut.lan_abduction(df_abst, Ghl, hl_coeffs)

In [14]:
LLmodels = {}
for iota in Ill:
    LLmodels[iota] = lanm.LinearAddSCM(Gll, ll_coeffs, iota)
    
HLmodels, Dhl_samples = {}, {}
for eta in Ihl:
    HLmodels[eta] = lanm.LinearAddSCM(Ghl, hl_coeffs, eta)

In [15]:
L_matrices = oput.compute_struc_matrices(LLmodels, Ill)
H_matrices = oput.compute_struc_matrices(HLmodels, Ihl)

In [16]:
def compute_empirical_radius(N, eta, c1=1.0, c2=1.0, alpha=2.0, m=3):
    """
    Compute epsilon_N(eta) for empirical Wasserstein case.

    Parameters:
    - N: int, number of samples
    - eta: float, confidence level (0 < eta < 1)
    - c1: float, constant from theorem (default 1.0, adjust if needed)
    - c2: float, constant from theorem (default 1.0, adjust if needed)
    - alpha: float, light-tail exponent (P[exp(||ξ||^α)] ≤ A)
    - m: int, ambient dimension

    Returns:
    - epsilon: float, the concentration radius
    """
    assert 0 < eta < 1, "eta must be in (0,1)"
    threshold = np.log(c1 / eta) / c2
    if N >= threshold:
        exponent = min(1/m, 0.5)
    else:
        exponent = 1 / alpha

    epsilon = (np.log(c1 / eta) / (c2 * N)) ** exponent
    return epsilon

In [17]:
ll_bound = round(compute_empirical_radius(N=num_llsamples, eta=0.05, c1=1000.0, c2=1.0, alpha=2.0, m=l), 3)
hl_bound = round(compute_empirical_radius(N=num_hlsamples, eta=0.05, c1=1000.0, c2=1.0, alpha=2.0, m=h), 3)


In [49]:
epsilon, delta = ll_bound, hl_bound

eta_max = 0.001
eta_min = 0.001

max_iter = 1000
num_steps_min = 5
num_steps_max = 5

robust_L = True 
robust_H = True

initialization = 'random'

tol  = 1e-5
seed = 23

In [50]:
opt_params_erica = {
                        'U_L': U_ll_hat,
                        'U_H': U_hl_hat,
                        'L_models': LLmodels,
                        'H_models': HLmodels,
                        'omega': omega,
                        'epsilon': epsilon,
                        'delta': delta,
                        'eta_min': eta_min,
                        'eta_max': eta_max,
                        'num_steps_min': num_steps_min,
                        'num_steps_max': num_steps_max,
                        'max_iter': max_iter,
                        'tol': tol,
                        'seed': seed,
                        'robust_L': robust_L,
                        'robust_H': robust_H,
                        'initialization': initialization,
                        'experiment': 'battery_discrete'
                    }

In [51]:
# Define different epsilon=delta values
eps_delta_values     = [8, ll_bound, 1, 2, 4]
diroca_train_results_empirical = {}

# For each epsilon=delta value
for eps_delta in eps_delta_values:
    print(f"Training for ε=δ = {eps_delta}")
    # Update theta parameters
    if eps_delta == ll_bound:
        opt_params_erica['epsilon'] = ll_bound
        opt_params_erica['delta']   = hl_bound
    
    else:
        opt_params_erica['epsilon'] = eps_delta
        opt_params_erica['delta']   = eps_delta
    
    # Run ERICA optimization
    params_empirical, T_empirical = oput.run_empirical_erica_optimization(**opt_params_erica)
    
    # Store results including optimization parameters and transformation matrix
    if eps_delta == ll_bound:
        diroca_train_results_empirical['T_'+str(ll_bound)+'-'+str(hl_bound)] = {
                                                    'optimization_params': params_empirical,
                                                    'T_matrix': T_empirical
                                                }
    else:
        diroca_train_results_empirical['T_'+str(eps_delta)] = {
                                                    'optimization_params': params_empirical,
                                                    'T_matrix': T_empirical
                                                }

print("\nTraining completed. T matrices stored in trained_results dictionary.")
print("Available ε=δ values:", list(diroca_train_results_empirical.keys()))



Training for ε=δ = 8


  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:14<00:00, 70.73it/s]


Training for ε=δ = 0.537


100%|██████████| 1000/1000 [00:15<00:00, 66.06it/s]


...

Converged at iteration 833
Training for ε=δ = 2


100%|██████████| 1000/1000 [00:14<00:00, 68.51it/s]


Training for ε=δ = 4


100%|██████████| 1000/1000 [00:14<00:00, 70.56it/s]


Training completed. T matrices stored in trained_results dictionary.
Available ε=δ values: ['T_8', 'T_0.537-0.393', 'T_1', 'T_2', 'T_4']


In [52]:
params_enrico, T_enrico = oput.run_empirical_erica_optimization(**{**opt_params_erica, 'robust_L': False, 'robust_H': False})

100%|██████████| 1000/1000 [00:01<00:00, 850.93it/s]


In [53]:
diroca_train_results_empirical['T_0.00'] = {
                                'optimization_params': params_enrico,
                                'T_matrix': T_enrico
                            }

In [54]:
opt_params_bary = {
                        'U_ll_hat':U_ll_hat,
                        'U_hl_hat':U_hl_hat,
                        'L_matrices':L_matrices,
                        'H_matrices':H_matrices,
                        'max_iter':1000,
                        'tol':tol,
                        'seed':seed
                    }
                                 

In [55]:
T_bary = oput.run_empirical_bary_optim(**opt_params_bary)
params_bary = {'L':{}, 'H':{}}

100%|██████████| 1000/1000 [00:00<00:00, 3410.31it/s]


In [56]:
diroca_train_results_empirical['T_b'] = {
                                'optimization_params': params_bary,
                                'T_matrix': T_bary
                            }

In [57]:
opt_params_smooth = {
                        'U_L': U_ll_hat,
                        'U_H': U_hl_hat,
                        'L_models': LLmodels,
                        'H_models': HLmodels,
                        'omega': omega,
                        'eta_min': eta_min,
                        'num_steps_min': num_steps_min,
                        'max_iter': 300,
                        'tol': tol,
                        'seed': seed,
                        'noise_sigma': 0.1,
                        'num_noise_samples': 10
                        }

In [58]:
params_smooth, T_smooth = oput.run_empirical_smooth_optimization(**opt_params_smooth)

100%|██████████| 300/300 [00:10<00:00, 27.54it/s]


In [59]:
diroca_train_results_empirical['T_s'] = {
                                'optimization_params': params_smooth,
                                'T_matrix': T_smooth
                            }

In [60]:
linabs_results = evut.run_abs_lingam_complete(df_base, df_abst)

In [61]:
diroca_train_results_empirical['T_pa'] = {'optimization_params':{'L':{'pert_U':U_ll_hat},'H':{'pert_U':U_hl_hat}}, 'T_matrix': linabs_results['Perfect']['T'].T}
diroca_train_results_empirical['T_na'] = {'optimization_params':{'L':{'pert_U':U_ll_hat},'H':{'pert_U':U_hl_hat}}, 'T_matrix': linabs_results['Noisy']['T'].T}

In [62]:
experiment = 'battery_discrete'

In [63]:
joblib.dump(diroca_train_results_empirical, f"data/{experiment}/diroca_train_results_empirical.pkl")

['data/battery_discrete/diroca_train_results_empirical.pkl']

In [64]:
# def downstream_evaluation(T, df_base, df_abst, noise_level, noise_in):
#     from sklearn.linear_model import Lasso
#     from sklearn.model_selection import LeaveOneGroupOut
#     from sklearn.metrics import mean_squared_error
#     import numpy as np

#     from sklearn.linear_model import Lasso
#     from sklearn.model_selection import LeaveOneGroupOut
#     from sklearn.metrics import mean_squared_error
#     import numpy as np

#     # Define fixed hyperparameters
#     lasso_params = {'alpha': 0.1, 'max_iter': 1000, 'tol': 0.01}
    
#     # Copy inputs
#     df_base_noisy = df_base.copy()
#     df_abst_noisy = df_abst.copy()
#     df_abst_noisy = df_abst_noisy.astype(float)

#     if noise_in == 'both':
#         df_base_noisy += np.random.normal(0, noise_level, df_base.shape)
#         df_abst_noisy += np.random.normal(0, noise_level, df_abst.shape)
#     elif noise_in == 'base':
#         df_base_noisy += np.random.normal(0, noise_level, df_base.shape)
#         df_abst_noisy = df_abst
#     elif noise_in =='abst':
#         df_abst_noisy += np.random.normal(0, noise_level, df_abst.shape)
#         df_base_noisy = df_base
#     elif noise_in == 'none':
#         df_base_noisy, df_abst_noisy = df_base, df_abst
    

#     # Generate transformed and real abstract samples
#     tau_samples = T @ df_base_noisy.T
#     abst_samples = df_abst_noisy.T

#     # Step 1: Transpose to get (N, dim)
#     X_real = abst_samples.T
#     X_gen  = tau_samples.T

#     # Step 2: Define target labels and intervention groupings
#     y = df_abst[:, 1]
#     groups = df_abst[:, 0]
#     # y = df_abst_noisy[:, "binned ML"]
#     # groups = df_abst_noisy[:, "Comma gap (µm)"]


#     assert X_real.shape[0] == len(y) == len(groups), "Mismatch in number of samples"

#     # Step 3: Combine real and generated data
#     X_all = np.concatenate([X_real, X_gen], axis=0)
#     y_all = np.concatenate([y, y], axis=0)
#     groups_all  = np.concatenate([groups, groups], axis=0)

#     logo = LeaveOneGroupOut()

#     # Mode 1: Real → Real
#     mse_real = []
#     for train_idx, test_idx in logo.split(X_real, y, groups=groups):
#         model = Lasso().fit(X_real[train_idx], y[train_idx])
#         y_pred = model.predict(X_real[test_idx])
#         mse_real.append(mean_squared_error(y[test_idx], y_pred))

#    # Mode 2: Augmented → Real
#     mse_aug = []
#     for train_idx, test_idx in logo.split(X_real, y, groups=groups):
#         # Use noisy y for training with transformed data
#         y_train = df_abst_noisy[train_idx, 1]
#         model = Lasso().fit(X_gen[train_idx], y_train)
#         y_pred = model.predict(X_real[test_idx])
#         mse_aug.append(mean_squared_error(y[test_idx], y_pred))

#     # Mode 3: Real + Augmented → Real
#     mse_mix = []
#     for test_group in np.unique(groups):
#         test_mask = (groups == test_group)
#         test_idx_real = np.where(test_mask)[0]

#         train_mask_real = (groups != test_group)
#         train_idx_real = np.where(train_mask_real)[0]
#         train_idx_gen = np.arange(len(y)) + len(y)
#         train_idx_all = np.concatenate([train_idx_real, train_idx_gen])

#         model = Lasso(**lasso_params).fit(X_all[train_idx_all], y_all[train_idx_all])

#         y_pred = model.predict(X_real[test_idx_real])
#         mse_mix.append(mean_squared_error(y[test_idx_real], y_pred))

#     return {
#         "Real": (np.mean(mse_real), np.std(mse_real)),
#         "Aug": (np.mean(mse_aug), np.std(mse_aug)),
#         "AugReal": (np.mean(mse_mix), np.std(mse_mix))
#     }

In [65]:
def downstream_evaluation_paper(T, df_base, df_abst):
    """
    Implements the paper's evaluation methodology with three scenarios
    """
    # Get unique Comma Gap values
    comma_gaps = np.unique(df_abst[:, 0])
    
    # Scenario 1: Before abstraction (Real → Real)
    mse_real = []
    for cg in comma_gaps:
        # Split data
        test_mask = (df_abst[:, 0] == cg)
        train_mask = ~test_mask
        
        # Train data
        X_train = df_abst[train_mask, 0].reshape(-1, 1)  # Comma Gap as feature
        y_train = df_abst[train_mask, 1]  # Mass Loading as target
        
        # Test data
        X_test = df_abst[test_mask, 0].reshape(-1, 1)
        y_test = df_abst[test_mask, 1]
        
        # Train and evaluate
        model = Lasso().fit(X_train, y_train)
        y_pred = model.predict(X_test)
        mse_real.append(mean_squared_error(y_test, y_pred))
    
    # Scenario 2: After abstraction with support (Aug → Real)
    mse_aug = []
    # Generate transformed samples
    tau_samples = T @ df_base.T
    enhanced_data = np.concatenate([df_abst, tau_samples.T])
    
    for cg in comma_gaps:
        test_mask = (df_abst[:, 0] == cg)
        train_mask_abst = ~test_mask
        train_mask_full = np.concatenate([train_mask_abst, np.ones(len(tau_samples.T), dtype=bool)])
        
        # Train data (including transformed samples)
        X_train = enhanced_data[train_mask_full, 0].reshape(-1, 1)
        y_train = enhanced_data[train_mask_full, 1]
        
        # Test data (only original LRCS)
        X_test = df_abst[test_mask, 0].reshape(-1, 1)
        y_test = df_abst[test_mask, 1]
        
        model = Lasso().fit(X_train, y_train)
        y_pred = model.predict(X_test)
        mse_aug.append(mean_squared_error(y_test, y_pred))
    
    # Scenario 3: After abstraction without support (Real+Aug → Real)
    mse_mix = []
    for cg in comma_gaps:
        test_mask_abst = (df_abst[:, 0] == cg)
        test_mask_tau = (tau_samples.T[:, 0] == cg)
        
        train_mask_abst = ~test_mask_abst
        train_mask_tau = ~test_mask_tau
        
        # Combine masks for training
        train_data = np.concatenate([
            df_abst[train_mask_abst],
            tau_samples.T[train_mask_tau]
        ])
        
        X_train = train_data[:, 0].reshape(-1, 1)
        y_train = train_data[:, 1]
        
        # Test only on LRCS data
        X_test = df_abst[test_mask_abst, 0].reshape(-1, 1)
        y_test = df_abst[test_mask_abst, 1]
        
        model = Lasso().fit(X_train, y_train)
        y_pred = model.predict(X_test)
        mse_mix.append(mean_squared_error(y_test, y_pred))
    
    return {
        "Real": (np.mean(mse_real), np.std(mse_real)),
        "Aug": (np.mean(mse_aug), np.std(mse_aug)),
        "AugReal": (np.mean(mse_mix), np.std(mse_mix))
    }

In [66]:
print("\n" + "="*80)
print(f"{'Method':<15} {'Real':<25} {'Aug':<25} {'AugReal':<25}")
# print(f"{'Method':<15} {'Real':<25} ")

print("="*80)

for method in list(diroca_train_results_empirical.keys()):
    T = diroca_train_results_empirical[method]['T_matrix']
    d = downstream_evaluation_paper(T, df_base, df_abst)#, noise_level=0.0, noise_in='both')
    
    real_str = f"{d['Real'][0]:.3f} ± {d['Real'][1]:.3f}"
    aug_str = f"{d['Aug'][0]:.3f} ± {d['Aug'][1]:.3f}"
    augreal_str = f"{d['AugReal'][0]:.3f} ± {d['AugReal'][1]:.3f}"
    
    print(f"{method:<15} {real_str:<25} {aug_str:<25} {augreal_str:<25}")
    # print(f"{method:<15} {real_str:<25} ")

print("="*80)


Method          Real                      Aug                       AugReal                  
T_8             5.556 ± 2.388             3.144 ± 2.736             3.144 ± 2.736            
T_0.537-0.393   5.556 ± 2.388             4.840 ± 6.185             4.840 ± 6.185            
T_1             5.556 ± 2.388             6.949 ± 9.273             6.949 ± 9.273            
T_2             5.556 ± 2.388             5.288 ± 7.175             5.288 ± 7.175            
T_4             5.556 ± 2.388             2.907 ± 1.907             2.907 ± 1.907            
T_0.00          5.556 ± 2.388             6.031 ± 5.746             6.031 ± 5.746            
T_b             5.556 ± 2.388             7.029 ± 5.610             7.029 ± 5.610            
T_s             5.556 ± 2.388             4.584 ± 3.946             4.584 ± 3.946            
T_pa            5.556 ± 2.388             3.310 ± 2.536             3.310 ± 2.536            
T_na            5.556 ± 2.388             3.310 ± 2.536    

In [67]:
def downstream_evaluation_paper_with_noise(T, df_base, df_abst, noise_level=0.0, noise_in='none'):
    """
    Implements the paper's evaluation methodology with three scenarios, with added noise functionality
    """
    # Create noisy copies of the data and convert to float
    df_base_noisy = df_base.copy().astype(float)
    df_abst_noisy = df_abst.copy().astype(float)
    
    # Add noise according to specified parameters
    if noise_in == 'both':
        df_base_noisy += np.random.normal(0, noise_level, df_base.shape)
        df_abst_noisy += np.random.normal(0, noise_level, df_abst.shape)
    elif noise_in == 'base':
        df_base_noisy += np.random.normal(0, noise_level, df_base.shape)
        df_abst_noisy = df_abst.copy().astype(float)
    elif noise_in == 'abst':
        df_abst_noisy += np.random.normal(0, noise_level, df_abst.shape)
        df_base_noisy = df_base.copy().astype(float)
    
    # Get unique Comma Gap values
    comma_gaps = np.unique(df_abst_noisy[:, 0])
    
    # Rest of the function remains the same...
    # Scenario 1: Before abstraction (Real → Real)
    mse_real = []
    for cg in comma_gaps:
        test_mask = (df_abst_noisy[:, 0] == cg)
        train_mask = ~test_mask
        
        X_train = df_abst_noisy[train_mask, 0].reshape(-1, 1)
        y_train = df_abst_noisy[train_mask, 1]
        
        X_test = df_abst_noisy[test_mask, 0].reshape(-1, 1)
        y_test = df_abst_noisy[test_mask, 1]
        
        model = Lasso().fit(X_train, y_train)
        y_pred = model.predict(X_test)
        mse_real.append(mean_squared_error(y_test, y_pred))
    
    # Scenario 2: After abstraction with support (Aug → Real)
    mse_aug = []
    # Generate transformed samples using noisy base data
    tau_samples = T @ df_base_noisy.T
    enhanced_data = np.concatenate([df_abst_noisy, tau_samples.T])
    
    for cg in comma_gaps:
        test_mask = (df_abst_noisy[:, 0] == cg)
        train_mask_abst = ~test_mask
        train_mask_full = np.concatenate([train_mask_abst, np.ones(len(tau_samples.T), dtype=bool)])
        
        X_train = enhanced_data[train_mask_full, 0].reshape(-1, 1)
        y_train = enhanced_data[train_mask_full, 1]
        
        X_test = df_abst_noisy[test_mask, 0].reshape(-1, 1)
        y_test = df_abst_noisy[test_mask, 1]
        
        model = Lasso().fit(X_train, y_train)
        y_pred = model.predict(X_test)
        mse_aug.append(mean_squared_error(y_test, y_pred))
    
    # Scenario 3: After abstraction without support (Real+Aug → Real)
    mse_mix = []
    for cg in comma_gaps:
        test_mask_abst = (df_abst_noisy[:, 0] == cg)
        test_mask_tau = (tau_samples.T[:, 0] == cg)
        
        train_mask_abst = ~test_mask_abst
        train_mask_tau = ~test_mask_tau
        
        train_data = np.concatenate([
            df_abst_noisy[train_mask_abst],
            tau_samples.T[train_mask_tau]
        ])
        
        X_train = train_data[:, 0].reshape(-1, 1)
        y_train = train_data[:, 1]
        
        X_test = df_abst_noisy[test_mask_abst, 0].reshape(-1, 1)
        y_test = df_abst_noisy[test_mask_abst, 1]
        
        model = Lasso().fit(X_train, y_train)
        y_pred = model.predict(X_test)
        mse_mix.append(mean_squared_error(y_test, y_pred))
    
    return {
        "Real": (np.mean(mse_real), np.std(mse_real)),
        "Aug": (np.mean(mse_aug), np.std(mse_aug)),
        "AugReal": (np.mean(mse_mix), np.std(mse_mix))
    }

In [68]:
print("\n" + "="*80)
print(f"{'Method':<15} {'Real':<25} {'Aug':<25} {'AugReal':<25}")
# print(f"{'Method':<15} {'Real':<25} ")

print("="*80)

for method in list(diroca_train_results_empirical.keys()):
    T = diroca_train_results_empirical[method]['T_matrix']
    d = downstream_evaluation_paper_with_noise(T, df_base, df_abst, noise_level=0.1, noise_in='both')
    
    real_str = f"{d['Real'][0]:.3f} ± {d['Real'][1]:.3f}"
    aug_str = f"{d['Aug'][0]:.3f} ± {d['Aug'][1]:.3f}"
    augreal_str = f"{d['AugReal'][0]:.3f} ± {d['AugReal'][1]:.3f}"
    
    print(f"{method:<15} {real_str:<25} {aug_str:<25} {augreal_str:<25}")
    # print(f"{method:<15} {real_str:<25} ")

print("="*80)


Method          Real                      Aug                       AugReal                  
T_8             1.361 ± 2.088             2.350 ± 1.129             2.350 ± 1.129            
T_0.537-0.393   1.349 ± 2.015             4.167 ± 2.334             4.167 ± 2.334            
T_1             1.403 ± 2.069             1.239 ± 0.709             1.239 ± 0.709            
T_2             1.369 ± 2.036             0.693 ± 0.420             0.693 ± 0.420            
T_4             1.367 ± 2.075             1.928 ± 0.980             1.928 ± 0.980            
T_0.00          1.396 ± 2.104             4.069 ± 2.134             4.069 ± 2.134            
T_b             1.312 ± 2.058             3.626 ± 1.656             3.626 ± 1.656            
T_s             1.349 ± 2.005             3.047 ± 1.507             3.047 ± 1.507            
T_pa            1.374 ± 1.992             1.353 ± 1.921             1.353 ± 1.921            
T_na            1.386 ± 2.163             1.364 ± 2.137    

In [66]:
def sample_from_frobenius_ball(params, radius_type, boundary, rand_rad):
    """
    Initialize a matrix inside the Frobenius ball with ||X||_F^2 <= N*epsilon^2
    """
    num_samples, num_vars = params['pert_U'].shape
    if radius_type == 'worst_case':
        radius = params['radius_worst']

    elif radius_type == 'hat_case':
        radius = params['radius']

    elif radius_type == 'random':
        radius = rand_rad
        
    matrix           = np.random.randn(num_samples, num_vars)  
    squared_norm     = np.linalg.norm(matrix, 'fro')**2
    max_squared_norm = num_samples * radius**2
    
    if boundary == True:
        scaling_factor = np.sqrt(max_squared_norm / squared_norm) * np.random.rand(1)
    else:
        scaling_factor = np.sqrt(max_squared_norm / squared_norm)
    return matrix * scaling_factor

In [67]:
def generate_pertubation(pert_type, pert_level, experiment, rad=None):
    
    params = load_empirical_optimization_params(experiment, pert_level)
    
    if pert_type == 'sample_radius':
        P = sample_from_frobenius_ball(params, 'hat_case', boundary=False, rand_rad=rad)
    
    elif pert_type == 'sample_radius_worst':
        P = sample_from_frobenius_ball(params, 'worst_case', boundary=False, rand_rad=rad)

    elif pert_type == 'boundary_worst':
        P = params['pert_U']
        
    elif pert_type == 'boundary_hat':
        P = sample_from_frobenius_ball(params, 'hat_case', boundary=True, rand_rad=rad)

    elif pert_type == 'random_hat':
        P = sample_from_frobenius_ball(params, 'hat_case', boundary=False, rand_rad=rad)

    elif pert_type == 'random_worst':
        P = sample_from_frobenius_ball(params, 'worst_case', boundary=False, rand_rad=rad)

    elif pert_type == 'boundary_random_hat':
        P = sample_from_frobenius_ball(params, 'hat_case', boundary=True, rand_rad=rad)

    elif pert_type == 'boundary_random_worst':
        P = sample_from_frobenius_ball(params, 'worst_case', boundary=True, rand_rad=rad)

    elif pert_type == 'random_normal':
        P = np.random.randn(*params['pert_U'].shape)

    return P

In [68]:
def downstream_evaluation_with_structured_noise(
    T, df_base, df_abst,
    noise_type='gaussian',  # or 'frobenius'
    noise_level=0.1,
    noise_in='base',  # 'base', 'abst', or 'both'
    pert_type=None,
    experiment=None,
    radius_L=None,
    radius_H=None
):
    """
    Evaluate abstraction robustness with either Gaussian or Frobenius-ball structured noise.
    """

    from sklearn.linear_model import Lasso
    from sklearn.model_selection import LeaveOneGroupOut
    from sklearn.metrics import mean_squared_error
    import numpy as np

    df_base_noisy = df_base.copy().astype(float)
    df_abst_noisy = df_abst.copy().astype(float)

    if noise_type == 'gaussian':
        if noise_in in ['base', 'both']:
            df_base_noisy += np.random.normal(0, noise_level, df_base.shape)
        if noise_in in ['abst', 'both']:
            df_abst_noisy += np.random.normal(0, noise_level, df_abst.shape)

    elif noise_type == 'frobenius':

        if noise_in in ['base', 'both']:
            assert pert_type is not None and experiment is not None
            pert_L = generate_pertubation(pert_type, 'L', experiment, rad=radius_L)
            df_base_noisy += pert_L
        if noise_in in ['abst', 'both']:
            assert pert_type is not None and experiment is not None
            pert_H = generate_pertubation(pert_type, 'H', experiment, rad=radius_H)
            df_abst_noisy += pert_H

    tau_samples = T @ df_base_noisy.T
    abst_samples = df_abst_noisy.T

    X_real = abst_samples.T
    X_gen = tau_samples.T

    y = df_abst_noisy[:, 1]
    groups = df_abst_noisy[:, 0]

    assert X_real.shape[0] == len(y) == len(groups), "Mismatch in sample counts"

    X_all = np.concatenate([X_real, X_gen], axis=0)
    y_all = np.concatenate([y, y], axis=0)
    groups_all = np.concatenate([groups, groups], axis=0)

    logo = LeaveOneGroupOut()

    mse_real = []
    for train_idx, test_idx in logo.split(X_real, y, groups=groups):
        model = Lasso().fit(X_real[train_idx], y[train_idx])
        y_pred = model.predict(X_real[test_idx])
        mse_real.append(mean_squared_error(y[test_idx], y_pred))

    mse_aug = []
    for train_idx, test_idx in logo.split(X_real, y, groups=groups):
        model = Lasso().fit(X_gen[train_idx], y[train_idx])
        y_pred = model.predict(X_real[test_idx])
        mse_aug.append(mean_squared_error(y[test_idx], y_pred))

    mse_mix = []
    for test_group in np.unique(groups):
        test_mask = (groups == test_group)
        test_idx_real = np.where(test_mask)[0]

        train_mask_real = (groups != test_group)
        train_idx_real = np.where(train_mask_real)[0]
        train_idx_gen = np.arange(len(y)) + len(y)
        train_idx_all = np.concatenate([train_idx_real, train_idx_gen])

        model = Lasso().fit(X_all[train_idx_all], y_all[train_idx_all])
        y_pred = model.predict(X_real[test_idx_real])
        mse_mix.append(mean_squared_error(y[test_idx_real], y_pred))

    return {
        "Real→Real": (np.mean(mse_real), np.std(mse_real)),
        "Aug→Real": (np.mean(mse_aug), np.std(mse_aug)),
        "Real+Aug→Real": (np.mean(mse_mix), np.std(mse_mix))
    }


In [69]:
# Gaussian noise in base-level only
downstream_evaluation_with_structured_noise(T, df_base, df_abst, noise_type='gaussian', noise_level=0.5, noise_in='base')

# Frobenius-ball structured noise (requires experiment + params)
downstream_evaluation_with_structured_noise(
    T, df_base, df_abst,
    noise_type='frobenius',
    noise_in='both',
    pert_type='random_worst',
    experiment='MyExperiment',
    radius_L=3,
    radius_H=2
)


NameError: name 'load_empirical_optimization_params' is not defined